In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import itertools

In [ ]:
tools_dir_path = os.path.abspath('')
root_dir_path = os.path.dirname(tools_dir_path)
print(root_dir_path)
results_dir_path = os.path.join(root_dir_path, 'results')

In [ ]:
# cur_data_dir_name = 'data-3990x'
# cur_data_dir_name = 'm1-max-202207'
# cur_data_dir_name = 'E3-1270v6'
# cur_data_dir_name = 'tempdata'
# cur_data_dir_name = 'data-xeon-6144'
cur_data_dir_name = 'm1-max'
# cur_data_dir_name = 'pi4'
cur_data_dir_path = os.path.join(results_dir_path, cur_data_dir_name)
cur_csv_dir_path = os.path.join(cur_data_dir_path, 'csv')
export_fig_dir_path = os.path.join(cur_data_dir_path, 'figs')
if not os.path.exists(export_fig_dir_path):
    os.mkdir(export_fig_dir_path)

In [ ]:
filename_list = [f for f in os.listdir(cur_csv_dir_path) if os.path.isfile(os.path.join(cur_csv_dir_path, f))]
data_filename_pattern = re.compile(r'(.+)__(.+)__(.+)__(.+)\.csv')
data_filename_list = []
data_filepath_list = []
map_name_list = []
hash_name_list = []
key_name_list = []
value_name_list = []
k_v_dict = {}
for filename in filename_list:
    match_ret = data_filename_pattern.findall(filename)
    if match_ret:
        data_file_path = os.path.join(cur_csv_dir_path, filename)
        data_filepath_list.append(data_file_path)
        data_filename_list.append(filename)
        temp_map_name = match_ret[0][0]
        temp_hash_name = match_ret[0][1]
        map_name_list.append(temp_map_name)
        hash_name_list.append(temp_hash_name)
        key_name_list.append(match_ret[0][2])
        value_name_list.append(match_ret[0][3])
        temp_k_v_name = '<' + match_ret[0][2] + ',' + match_ret[0][3] + '>'
        # k_v_name_list.append(temp_k_v_name)
        if not temp_k_v_name in k_v_dict:
            k_v_dict[temp_k_v_name] = {'data_filepath_list':[], 'hash_name_list':[], 'map_name_list':[]}
        k_v_dict[temp_k_v_name]['data_filepath_list'].append(data_file_path)
        k_v_dict[temp_k_v_name]['hash_name_list'].append(temp_hash_name)
        k_v_dict[temp_k_v_name]['map_name_list'].append(temp_map_name)
print("total %d data files" % (len(data_filename_list)))
map_name_set = set(map_name_list)
hash_name_set = set(hash_name_list)
print(data_filename_list)
print(map_name_set)
print(hash_name_set)
print(k_v_dict.keys())

In [ ]:
import matplotlib.ticker as plticker
from matplotlib.ticker import FormatStrFormatter
def plot_lines(title, x_arr_list, y_arr_list, label_list, y_axis_label, ymax=None, save_fig=False, show_fig=False, export_dir_path=None, save_name=None):
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    fig.suptitle(title, fontsize='small')
    assert(len(x_arr_list) == len(y_arr_list))
    arr_num = len(x_arr_list)
    marker = itertools.cycle(('v','o','^','<','>','s','p','*','+','x','D')) 
    for i in range(arr_num):
        plot_num = 0
        for j in range(len(y_arr_list[i])):
            if y_arr_list[i][j] == 0.0:
                break
            plot_num += 1
        ax.plot(x_arr_list[i][:plot_num], y_arr_list[i][:plot_num], label=label_list[i], marker=next(marker), markersize=2.5, linewidth=1.0)
    ax.set_xlabel('element num')
    ax.set_ylabel(y_axis_label)
    ax.set_yscale('log')
    ax.set_xscale('log')
    default_ymin, default_ymax = ax.get_ylim()
    if not ymax is None:
        if ymax < default_ymax:
            ax.set_ylim([default_ymin, ymax])
    subs = [1.0, 2.0, 4.0, 6.0]
    ax.yaxis.set_minor_locator(plticker.LogLocator(subs=subs))
    ax.yaxis.set_minor_formatter(FormatStrFormatter("%.0f"))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    fig.legend(loc='center right', fontsize='xx-small')
    
    if save_fig:
        export_fig_name = title + '.png'
        if not save_name is None:
            export_fig_name = save_name
        export_fig_path = os.path.join(export_dir_path, export_fig_name)
        fig.savefig(export_fig_path, transparent=False, facecolor='white')
    # if you need to show figures in the notebook, use plt.show()
    if show_fig:
        plt.show()
    plt.close(fig)
    

In [ ]:
# whether show the figure in notebook
show_figure = False
# whether save the figure to picture files
save_figure = True
for k_v_name, data_dict in k_v_dict.items():
    # print(k_v_name)
    data_file_num = len(data_dict['data_filepath_list'])
    map_hash_name_list = []
    element_num_arr_list = []
    construct_time_with_reserve_arr_list = []
    construct_time_without_reserve_arr_list = []
    erase_insert_arr_list = []
    hit_without_rehash_lookup_arr_list = []
    miss_without_rehash_lookup_arr_list = []
    may_without_rehash_lookup_arr_list = []
    hit_with_rehash_lookup_arr_list = []
    miss_with_rehash_lookup_arr_list = []
    may_with_rehash_lookup_arr_list = []
    iterate_arr_list = []
    with_final_rehash_construct_arr_list = []
    latency_arr_list_dict = {}
    for t in range(data_file_num):
        temp_map_hash_name = data_dict['map_name_list'][t] + "," + data_dict['hash_name_list'][t]
        map_hash_name_list.append(temp_map_hash_name)
        temp_data_filepath = data_dict['data_filepath_list'][t]
        temp_data_df = pd.read_csv(temp_data_filepath, sep=',')
        temp_data_df = temp_data_df.rename(columns=lambda x: x.strip())
        column_name_list = temp_data_df.columns.to_list()
        latency_item_pattern = re.compile(r'.+_latency')
        for column_name in column_name_list:
            match_obj = latency_item_pattern.match(column_name)
            if match_obj is not None:
                if column_name not in latency_arr_list_dict:
                    latency_arr_list_dict[column_name] = []
                latency_arr_list_dict[column_name].append(temp_data_df[column_name])
        element_num_arr_list.append(temp_data_df['element_num'])
        construct_time_with_reserve_arr_list.append(temp_data_df['avg_construct_time_with_reserve_ns'])
        construct_time_without_reserve_arr_list.append(temp_data_df['avg_construct_time_without_reserve_ns'])
        erase_insert_arr_list.append(temp_data_df['avg_erase_insert_ns'])
        hit_without_rehash_lookup_arr_list.append(temp_data_df['avg_hit_default_load_factor_lookup_ns'])
        miss_without_rehash_lookup_arr_list.append(temp_data_df['avg_miss_default_load_factor_lookup_ns'])
        may_without_rehash_lookup_arr_list.append(temp_data_df['avg_50%_hit_default_load_factor_lookup_ns'])
        hit_with_rehash_lookup_arr_list.append(temp_data_df['avg_hit_large_max_load_factor_lookup_ns'])
        miss_with_rehash_lookup_arr_list.append(temp_data_df['avg_miss_large_max_load_factor_lookup_ns'])
        may_with_rehash_lookup_arr_list.append(temp_data_df['avg_50%_hit_large_max_load_factor_lookup_ns'])
        iterate_arr_list.append(temp_data_df['avg_iterate_ns'])
        with_final_rehash_construct_arr_list.append(temp_data_df['avg_with_final_rehash_construct_ns'])
    plot_lines(k_v_name + ',avg_insert_time_with_reserve', element_num_arr_list, construct_time_with_reserve_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_insert_time_without_reserve', element_num_arr_list, construct_time_without_reserve_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, show_fig=show_figure)
    plot_lines(k_v_name +',avg_erase_insert_time', element_num_arr_list, erase_insert_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_hit_find_without_rehash', element_num_arr_list, hit_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_miss_find_without_rehash', element_num_arr_list, miss_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + r',avg_50%_hit_without_rehash', element_num_arr_list, may_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_hit_find_with_rehash', element_num_arr_list, hit_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_miss_find_with_rehash', element_num_arr_list, miss_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + r',avg_50%_hit_with_rehash', element_num_arr_list, may_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure)
    plot_lines(k_v_name + ',avg_iterate', element_num_arr_list, iterate_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path)
    plot_lines(k_v_name + ',avg_construct_time_with_final_rehash', element_num_arr_list, with_final_rehash_construct_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path)
    for latency_item, data_arr_list in latency_arr_list_dict.items():
        plot_lines(k_v_name + ',' + latency_item, element_num_arr_list, data_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_dir_path=export_fig_dir_path, show_fig=show_figure)
